In [ ]:
import sys
sys.path.append('./models')
from models.BBSNet_model import BBSNetTransformerAttention
from models.BBSNet_model import BBSNetChannelSpatialAttention
from BBSNet_train import setup_gpu, get_model, get_data, main
from BBSNet_test import test
from utils import evaluate, draw_curve

%load_ext autoreload
%autoreload 2

In [ ]:
# from models.RFNet import RFNet
# import torch

# rgb = torch.rand((2, 3, 352, 352)).cuda()
# depth = torch.rand((2, 1, 352, 352)).cuda()
# model = RFNet(32, 50).cuda()
# l = model(rgb, depth)

# Train

In [ ]:
setup_gpu()
model, optimizer, CE = get_model()
train_loader, test_loader, total_step = get_data()
main(optimizer, model, train_loader, test_loader, CE, total_step)

# Use Trained Model

I save the model using this code:
torch.save(model.state_dict(), save_path +
                           'BBSNet_epoch_best.pth')

Now, after a long training period, I want to use it on my test data. So, how can I load this model?

In [15]:
from models.BBSNet_model import BBSNetTransformerAttention

model = BBSNetTransformerAttention()

# assume your model is called `model`
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Total parameters: {total_params}")
print(f"Trainable parameters: {trainable_params}")
print(f"Non-trainable parameters: {total_params - trainable_params}")


Total parameters: 60894418
Trainable parameters: 60894418
Non-trainable parameters: 0


In [ ]:
# test(method='bbsnet') 

In [ ]:
from graphviz import Digraph

# Create a directed graph with horizontal layout
dot = Digraph()
dot.attr(rankdir='LR', dpi='300', fontname='Arial', nodesep='0.25', ranksep='0.35')  
dot.attr('node', shape='box', style='rounded,filled', fontname='Arial',
         height='0.5', width='1.2', fontsize='12', penwidth='1.2', color='black')

# Define nodes (clear & consistent labels)
dot.node('x', 'Input\n[B, C, H, W]', fillcolor='lightblue')
dot.node('max_pool', 'Global AvgPool\n[B, C, 1, 1]', fillcolor='lightgrey')
dot.node('fc1', '1×1 Conv\nC → C/16', fillcolor='orange')
dot.node('relu1', 'ReLU', fillcolor='darkseagreen2')
dot.node('fc2', '1×1 Conv\nC/16 → C', fillcolor='orange')
dot.node('sigmoid', 'Sigmoid', fillcolor='plum1')
dot.node('out', 'Output\n[B, C, 1, 1]', fillcolor='lightcoral')

# Connect nodes (short, clear arrows)
dot.edge('x', 'max_pool')
dot.edge('max_pool', 'fc1')
dot.edge('fc1', 'relu1')
dot.edge('relu1', 'fc2')
dot.edge('fc2', 'sigmoid')
dot.edge('sigmoid', 'out')

# Title at top
dot.attr(labelloc='t',fontsize='14', fontname='Arial Bold')

# Render and save
dot.render('channel_attn', format='png', cleanup=True)
print("Diagram generated as 'channel_attention_clean.png'")



Diagram generated as 'channel_attention_bw.png'


In [33]:
from graphviz import Digraph

# Create a directed graph with horizontal layout
dot = Digraph()
dot.attr(rankdir='LR', dpi='300', fontname='Arial', nodesep='0.25', ranksep='0.35')
dot.attr('node', shape='box', style='rounded,filled', fontname='Arial',
         height='0.5', width='1.2', fontsize='12', penwidth='1.2', color='black')

# Nodes with consistent labels and colors
dot.node('x', 'Input\n[B, C, H, W]', fillcolor='lightblue')
dot.node('max_pool', 'Max Pool\nChannel-wise\n[B, 1, H, W]', fillcolor='lightgrey')
dot.node('conv', 'Conv2d\n1→1, k=7', fillcolor='orange')
dot.node('sigmoid', 'Sigmoid', fillcolor='plum1')
dot.node('out', 'Output\n[B, 1, H, W]', fillcolor='lightcoral')

# Edges
dot.edge('x', 'max_pool')
dot.edge('max_pool', 'conv')
dot.edge('conv', 'sigmoid')
dot.edge('sigmoid', 'out')

# Title
dot.attr(labelloc='t',  fontsize='14', fontname='Arial Bold')

# Render and save
dot.render('spatial_attn', format='png', cleanup=True)
print("Diagram generated as 'spatial_attention_clean.png'")


Diagram generated as 'spatial_attention_clean.png'
